In [2]:
from datasets.dataset import FeatureLookup
import dgl as d
import pickle as pk
import numpy as np
import pandas as pd

class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.node2vec_windows = 3
        self.slices = 10

# node2vec
def eraser(datasets, args):
    label = None

    # try:
    # label = pk.load(open(f'./pretrain/user_based_{args.slices}.pk', 'rb'))
    # except IOError:

    userg = d.graph([])
    user_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()

    for i in range(339):
        user_lookup.register('User', i)

    for ure in ulines[:, 2]:
        user_lookup.register('URE', ure)

    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    userg.add_nodes(len(user_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])

        if not userg.has_edges_between(uid, ure):
            userg.add_edges(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edges_between(uid, uas):
            userg.add_edges(uid, uas)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)

    from node2vec import Node2Vec
    G = userg.to_networkx()

    # 设置node2vec参数
    node2vec = Node2Vec(
        G,
        dimensions=args.node2vec_dim,  # 嵌入维度
        p=1,  # 回家参数
        q=0.5,  # 外出参数
        walk_length=args.node2vec_length,  # 随机游走最大长度
        num_walks=args.node2vec_walk,  # 每个节点作为起始节点生成的随机游走个数
        workers=4  # 并行线程数
    )

    # p=1, q=0.5, n_clusters=6。DFS深度优先搜索，挖掘同质社群
    # p=1, q=2, n_clusters=3。BFS宽度优先搜索，挖掘节点的结构功能。

    # 训练Node2Vec，参数文档见 gensim.models.Word2Vec
    model = node2vec.fit(
        window=args.node2vec_windows,  # Skip-Gram窗口大小
        epochs=args.node2vec_epochs,
        min_count=1,  # 忽略出现次数低于此阈值的节点（词）
        batch_words=args.node2vec_batchsize  # 每个线程处理的数据量
    )
    ans = model.wv.vectors[:339]

    df = np.array(ans)

    # 整理特征
    pk.dump(df, open(f'./pretrain/user_embeds.pk', 'wb'))
    # return df

    def k_mean(inputs, k):
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(inputs)
        return kmeans.labels_

    label = k_mean(df, args.slices)
    pk.dump(label, open(f'./pretrain/user_based_{args.slices}.pk', 'wb'))

    # 检查每个类的个数
    dic = {}
    for i in label:
        if i not in dic:
            dic[i] = 1
        else:
            dic[i] = dic[i] + 1
    print(dic)
# node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize
args = a(128, 20, 50, 30, 32)
eraser(None, args)

Computing transition probabilities:   0%|          | 0/507 [00:00<?, ?it/s]

{0: 88, 1: 69, 6: 17, 5: 12, 7: 40, 8: 10, 3: 9, 2: 10, 4: 12, 9: 72}


In [3]:
with open('./datasets/data/WSDREAM/原始数据/userlist_table.csv', 'rb') as f:
    df = pd.read_csv(f)
with open('./pretrain/user_based_10.pk', 'rb') as f:
    label = pk.load(f)

C = [[] for _ in range(10)]
for i in range(10):
    for idx, typ in enumerate(label):
        if label[idx] == i:
            C[i].append([typ, df['[Country]'][idx], df['[AS]'][idx]])

In [4]:
df.columns

Index(['[User ID]', '[IP Address]', '[Country]', '[IP No.]', '[AS]',
       '[Latitude]', '[Longitude]'],
      dtype='object')

In [5]:
np.array(C[0])

array([['0', 'United States', 'AS7018 AT&T Services, Inc.'],
       ['0', 'United States', 'AS160 University of Chicago'],
       ['0', 'United States', 'AS6356 Northeast Regional Data Center'],
       ['0', 'United Kingdom', 'AS786 The JANET IP Service'],
       ['0', 'Canada', 'AS22950 University of Saskatchewan'],
       ['0', 'Canada', 'AS22950 University of Saskatchewan'],
       ['0', 'United States', 'AS156 Northeastern University'],
       ['0', 'United States', 'AS103 Northwestern University'],
       ['0', 'United States', 'AS18515 University of Texas at Arlington'],
       ['0', 'United States', 'AS2701 Kansas State University'],
       ['0', 'United States', 'AS3112 OARnet'],
       ['0', 'United States', 'AS4385 Rochester Institute of Technology'],
       ['0', 'Hong Kong', 'AS3661 The Chinese University of Hong Kong'],
       ['0', 'Spain', 'AS766 RedIRIS Autonomous System'],
       ['0', 'United States', 'AS209 Qwest Communications Company, LLC'],
       ['0', 'United Ki

In [6]:
np.array(C[1])

array([['1', 'United States', 'AS7018 AT&T Services, Inc.'],
       ['1', 'Japan', 'AS4713 NTT Communications Corporation'],
       ['1', 'United States', 'AS17 Purdue University'],
       ['1', 'United States',
        'AS131 University of California, Santa Barbara'],
       ['1', 'United States', 'AS88 Princeton University'],
       ['1', 'United States', 'AS88 Princeton University'],
       ['1', 'United States',
        'AS5739 University of California, Santa Cruz'],
       ['1', 'United States',
        'AS5739 University of California, Santa Cruz'],
       ['1', 'United States', 'AS1249 Five Colleges Network'],
       ['1', 'United States', 'AS160 University of Chicago'],
       ['1', 'United States', 'AS9 Carnegie Mellon University'],
       ['1', 'United States', 'AS5723 Johns Hopkins University'],
       ['1', 'United States', 'AS5723 Johns Hopkins University'],
       ['1', 'United States', 'AS2552 Washington University'],
       ['1', 'United States', 'AS14 Columbia Universi

In [7]:
np.array(C[2])

array([['2', 'United States', 'AS4201 Oregon State University'],
       ['2', 'United States', 'AS18 University of Texas at Austin'],
       ['2', 'United States', 'AS2496 University of Kansas'],
       ['2', 'Germany',
        'AS553 Landeshochschulnetz Baden-Wuerttemberg (BelWue)'],
       ['2', 'Germany', 'AS680 service G-WiN'],
       ['2', 'Germany', 'AS680 service G-WiN'],
       ['2', 'Italy', 'AS137 GARR Italian academic and research network'],
       ['2', 'Germany', 'AS680 service G-WiN'],
       ['2', 'United States',
        'AS22742 State of Connecticut Dept of InformationTechnology'],
       ['2', 'Austria', 'AS1110 University of Innsbruck']], dtype='<U58')

In [8]:
np.array(C[3])

array([['3', 'United States', 'AS1249 Five Colleges Network'],
       ['3', 'United States', 'AS14 Columbia University'],
       ['3', 'United States',
        'AS7377 University of California at San Diego'],
       ['3', 'Israel', 'AS378 ILAN'],
       ['3', 'United States',
        'AS11399 Williams College Center for Computing'],
       ['3', 'Austria', 'AS1110 University of Innsbruck'],
       ['3', 'Germany', 'AS12816 Leibniz-Rechenzentrum Muenchen'],
       ['3', 'Greece',
        'AS5408 Greek Research & Technology Network, http://www.grnet.gr'],
       ['3', 'United States',
        'AS25 University of California at Berkeley']], dtype='<U63')

In [9]:
np.array(C[4])

array([['4', 'United States', 'AS73 University of Washington'],
       ['4', 'United States', 'AS6356 Northeast Regional Data Center'],
       ['4', 'United States', 'AS14 Columbia University'],
       ['4', 'United States', 'AS46 Rutgers University'],
       ['4', 'Germany', 'AS8365 Man-da.de GmbH'],
       ['4', 'Switzerland',
        'AS559 SWITCH, Swiss Education and Research Network'],
       ['4', 'United States', 'AS22925 Allied Telecom Group, LLC'],
       ['4', 'Germany', 'AS680 service G-WiN'],
       ['4', 'United States', 'AS11318 Georgetown University'],
       ['4', 'United States', 'AS264 SRI International'],
       ['4', 'Czech Republic', 'AS2852 CESNET, z.s.p.o.'],
       ['4', 'Poland', 'AS5617 Telekomunikacja Polska S.A.']],
      dtype='<U50')

In [10]:
np.array(C[5])

array([['5', 'United States',
        'AS131 University of California, Santa Barbara'],
       ['5', 'United States', 'AS6510 Brigham Young University'],
       ['5', 'United States',
        'AS11399 Williams College Center for Computing'],
       ['5', 'Germany', 'AS680 service G-WiN'],
       ['5', 'Japan',
        'AS17932 Japan Advanced Institute of Science and Technology'],
       ['5', 'United States', 'AS13371 Duke University'],
       ['5', 'United States', 'AS6192 University of California at Davis'],
       ['5', 'Portugal',
        'AS1930 Rede Ciencia Tecnologia e Sociedade (RCTS)'],
       ['5', 'Poland', 'AS5617 Telekomunikacja Polska S.A.'],
       ['5', 'United States', 'AS292 ESnet'],
       ['5', 'Japan', 'AS2500 WIDE Project'],
       ['5', 'United States', 'AS4 University of Southern California']],
      dtype='<U58')

In [11]:
np.array(C[6])

array([['6', 'United States', 'AS17 Purdue University'],
       ['6', 'United States',
        'AS16461 The University of Texas at El Paso'],
       ['6', 'Germany', 'AS8365 Man-da.de GmbH'],
       ['6', 'United States', 'AS31 California Institute of Technology'],
       ['6', 'Germany', 'AS680 service G-WiN'],
       ['6', 'United States', 'AS5661 USF - UNIVERSITY OF SOUTH FLORIDA'],
       ['6', 'United States', 'AS5661 USF - UNIVERSITY OF SOUTH FLORIDA'],
       ['6', 'United States',
        'AS7377 University of California at San Diego'],
       ['6', 'Germany',
        'AS553 Landeshochschulnetz Baden-Wuerttemberg (BelWue)'],
       ['6', 'United States', 'AS11318 Georgetown University'],
       ['6', 'United States', 'AS36375 University of Michigan'],
       ['6', 'United States', 'AS237 Merit Network Inc.'],
       ['6', 'Greece', 'AS3323 National Technical University of Athens'],
       ['6', 'Greece',
        'AS5408 Greek Research & Technology Network, http://www.grnet.gr']

In [12]:
np.array(C[7])

array([['7', 'United States',
        'AS131 University of California, Santa Barbara'],
       ['7', 'United States', 'AS6510 Brigham Young University'],
       ['7', 'United States', 'AS73 University of Washington'],
       ['7', 'United States', 'AS73 University of Washington'],
       ['7', 'United States', 'AS2552 Washington University'],
       ['7', 'United States', 'AS2552 Washington University'],
       ['7', 'United States',
        'AS3 Massachusetts Institute of Technology'],
       ['7', 'United States', 'AS8 Rice University'],
       ['7', 'United States', 'AS8 Rice University'],
       ['7', 'United States',
        'AS27 University of Maryland at College Park'],
       ['7', 'United States', 'AS18 University of Texas at Austin'],
       ['7', 'United States', 'AS26 Cornell University'],
       ['7', 'United States', 'AS26 Cornell University'],
       ['7', 'United States', 'AS26 Cornell University'],
       ['7', 'United States', 'AS156 Northeastern University'],
       

In [13]:
np.array(C[8])

array([['8', 'United States',
        'AS5739 University of California, Santa Cruz'],
       ['8', 'Switzerland',
        'AS559 SWITCH, Swiss Education and Research Network'],
       ['8', 'United States',
        'AS52 University of California, Los Angeles'],
       ['8', 'United States', 'AS5661 USF - UNIVERSITY OF SOUTH FLORIDA'],
       ['8', 'United Kingdom', 'AS786 The JANET IP Service'],
       ['8', 'Germany', 'AS680 service G-WiN'],
       ['8', 'United States', 'AS22925 Allied Telecom Group, LLC'],
       ['8', 'Austria', 'AS1111 University of Klagenfurt'],
       ['8', 'United States', 'AS32 Stanford University'],
       ['8', 'Portugal',
        'AS1930 Rede Ciencia Tecnologia e Sociedade (RCTS)']],
      dtype='<U50')

In [14]:
np.array(C[9])

array([['9', 'United States', 'AS73 University of Washington'],
       ['9', 'United States', 'AS23329 Open Access Inc.'],
       ['9', 'United States', 'AS20162 University of Texas at Dallas'],
       ['9', 'United States', 'AS20162 University of Texas at Dallas'],
       ['9', 'United States', 'AS4385 Rochester Institute of Technology'],
       ['9', 'United States', 'AS7212 Vanderbilt University'],
       ['9', 'Puerto Rico', 'AS5786 University of Puerto Rico'],
       ['9', 'Singapore', 'AS7472 Computer Centre'],
       ['9', 'United States',
        'AS11399 Williams College Center for Computing'],
       ['9', 'Germany', 'AS12816 Leibniz-Rechenzentrum Muenchen'],
       ['9', 'Germany', 'AS680 service G-WiN'],
       ['9', 'Greece', 'AS8522 FORTH Autonomous System'],
       ['9', 'Taiwan', 'AS17716 National Taiwan University'],
       ['9', 'Taiwan', 'AS17716 National Taiwan University'],
       ['9', 'Taiwan',
        'AS1659 Tiawan Academic Network (TANet) Information Center'],